In [93]:
from Indicadores import df_indicadores
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os

In [94]:
df_indicadores

,SIGLA,INDICADOR,OBJETIVO,FÓRMULA,POLARIDADE,VALOR (Xi),PESO (Pi),Di
0,IND.01,INDICADOR DE SUCESSO DA PROPOSTA,PERCENTUAL DE SUCESSO DAS OBRAS PELAS AÇÕES EN...,TOTAL DE AÇÕES CANCELADAS / TOTAL DE AÇÕES CAD...,MENOR MELHOR,X1,NaN,NaN
1,IND.02,INDICADOR DE DOCUMENTAÇÃO INICIAL,ACOMPANHAR O PREENCHIMENTO DA DOCUMENTAÇÃO DE ...,TOTAL DE AÇÕES COM DOCUMENTAÇÃO / TOTAL DE AÇÕ...,MAIOR MELHOR,X2,NaN,NaN
2,IND.03,INDICADOR DE EVOLUÇÃO DAS AÇÕES,ACOMPANHAR AS AÇÕES CONSIDERADAS SEM EVOLUÇÃO ...,TOTAL DE AÇÕES SEM EVOLUÇÃO OU PARALISADAS COM...,MENOR MELHOR,X3,NaN,NaN
3,IND.04,INDICADOR DA EFICIÊNCIA FINANCEIRA,NaN,VALOR FINAL DO CONTRATO / VALOR ORIGINAL DO CO...,MENOR MELHOR,X4,NaN,NaN
4,IND.05,INDICADOR DE PAGAMENTO,PERCENTUAL DE AÇÕES SEM PAGAMENTO HÁ MAIS DE 9...,TOTAL DE AÇÕES MAIS DE 90D SEM PAGAMENTO / TOT...,MENOR MELHOR,X5,NaN,NaN
5,IND.06,INDICADOR DE VIGÊNCIA CONTRATUAL,NaN,TOTAL DE AÇÕES COM VIGÊNCIA EXPIRADA / TOTAL D...,MENOR MELHOR,X6,NaN,NaN
6,IND.07,INDICADOR DE AÇÕES EM EXECUÇÃO SEM ADITIVO,NaN,QUANT. DE AÇÕES EM EXECUÇÃO SEM ADITIVO / QUAN...,MENOR MELHOR,X7,NaN,NaN
7,IND.08,INDICADOR DA META ESTRATÉGICA,VERIFICAR O ANDAMENTOS DAS OBRAS CONFORME O PL...,TOTAL DE AÇÕES PLANEJADAS / TOTAL DE AÇÕES CON...,MAIOR MELHOR,X8,NaN,NaN
8,IND.09,INDICADOR DE DOCUMENTAÇÃO FINAL,ACOMPANHAMENTO DE AÇÕES COM A EMISSÃO DO TERMO...,TOTAL DE AÇÕES COM TRD OU TRP / TOTAL DE AÇÕES...,MAIOR MELHOR,X9,NaN,NaN
9,IND.GERAL,INDICADOR GERAL,ACOMPANHAMENTO DA EVOLUÇÃO PARA CADA SECRETARIA,ID.G.UG = ( 100 / PESO ) x ( SOMATÓRIO Di),MAIOR MELHOR,-,-,-


In [95]:
df_simo = pd.read_excel("tab_simo.xlsx")

In [78]:
df_simo['start_date'] = pd.to_datetime(df_simo['start_date'])
df_simo.at[1, "start_date"]

Timestamp('2023-05-18 00:00:00')

In [79]:
df_simo['Status'].unique()

array(['Em desenvolvimento', 'Não iniciada', nan, 'Cancelada',
       'Sem evolução', 'Concluído', 'Paralisado com Termo'], dtype=object)

In [80]:
df_simo.columns

Index(['Id_acao', 'Unidade_Gestora', 'Ação_Descricao', 'Prazo de conclusao',
       'Situação', 'Orçamento_previsto', 'Orçamento_total',
       'Numero SIAF do Convenio', 'Numero do Contrato', 'Tipo',
       'Classificação', 'Sub Classificacao da obra', 'TIPOLOGIA',
       'Numero do Contrato no SIAFE', 'Tags', 'Fim da Vigência', 'Link_SIMO',
       'Vencimento', 'Faixa Orçamento', 'Estagio_atual', 'Tipologia ajuste',
       '% Execução ação', 'Intervalo Execução', 'cód_Contrato', 'Integração',
       'ClassificacaoNumerica', 'Estagio Complementar',
       'siafe_contratos.valor_total', 'siafe_contratos.Pagamento',
       'action_projects__action_activities.Data Referência',
       'Valor medido Total', 'Suplementação', 'Ação Conveniada', 'start_date',
       'end_date', 'Convênio', 'Operação', 'PG', 'Especificação',
       'Filter - TAGS', 'Especificação 2', 'Fim do Contrato',
       'Detalhes_Vencimento', 'Status'],
      dtype='object')

In [96]:
# DATA DE REFERÊNCIA ~ Exatamente 5 anos atrás

hoje = datetime.now()
data_ref = hoje - timedelta (days= (5 * 365 + 1)) #

In [82]:
# INDICADOR 1
ind_01 = df_simo[df_simo['start_date'] >= pd.to_datetime(data_ref)].groupby(['Unidade_Gestora']).apply(lambda x: pd.Series({
    'Ações Canceladas': (x['Status'] == "Cancelada").sum(),
    'Total de Ações':  (x['Estagio_atual'] == "EM EXECUÇÃO").sum() # len(x)  # ou x['Status'].count()
}))

ind_01['Indicador 1'] = ind_01['Ações Canceladas'] / ind_01['Total de Ações']

ind_01.head(3)

,Ações Canceladas,Total de Ações,Indicador 1
Unidade_Gestora,,,
ADAPI,0,0,NaN
ADH,0,65,0.00
AGESPISA,2,25,0.08


In [83]:
# INDICADOR 3

ind_03 = df_simo[df_simo['start_date'] >= pd.to_datetime(data_ref)].groupby(['Unidade_Gestora']).apply(lambda x: pd.Series({
    'Ações sem evolução': (x['Status'] == "Sem evolução").sum(),
    'Ações paralisadas': (x['Status'] == "Paralisado com Termo").sum(),
    'Total de Ações':  x['Status'].count()
}))

ind_03['Indicador 3'] = (ind_03['Ações sem evolução'] + ind_03['Ações paralisadas']) / ind_03['Total de Ações']

ind_03.head(3)

,Ações sem evolução,Ações paralisadas,Total de Ações,Indicador 3
Unidade_Gestora,,,,
ADAPI,0,0,1,0.000000
ADH,7,0,86,0.081395
AGESPISA,3,1,30,0.133333


In [84]:
# INDICADOR 3

ind_03 = df_simo[df_simo['start_date'] >= pd.to_datetime(data_ref)].groupby(['Unidade_Gestora']).apply(lambda x: pd.Series({
    'Ações sem evolução': (x['Status'] == "Sem evolução").sum(),
    'Ações paralisadas': (x['Status'] == "Paralisado com Termo").sum(),
    'Total de Ações':  x['Status'].count()
}))

ind_03['Indicador 3'] = (ind_03['Ações sem evolução'] + ind_03['Ações paralisadas']) / ind_03['Total de Ações']

ind_03.head(3)

,Ações sem evolução,Ações paralisadas,Total de Ações,Indicador 3
Unidade_Gestora,,,,
ADAPI,0,0,1,0.000000
ADH,7,0,86,0.081395
AGESPISA,3,1,30,0.133333


In [ ]:
# INDICADOR 4

In [ ]:
# INDICADOR 5

In [85]:
# INDICADOR 6

df_simo = df_simo[df_simo['Estagio_atual'] == "EM EXECUÇÃO"]
ind_06 = df_simo[df_simo['start_date'] >= pd.to_datetime(data_ref)].groupby(['Unidade_Gestora']).apply(lambda x: pd.Series({
    'Ações Vigencia Expirada': (x['end_date'] < pd.to_datetime(hoje)).sum(),
    'Total de Ações':  (x['Estagio_atual'] == "EM EXECUÇÃO").sum() # len(x)  # ou x['Status'].count()
}))

ind_06['Indicador 6'] = ind_06['Ações Vigencia Expirada'] / ind_06['Total de Ações']

ind_06.head(4)

,Ações Vigencia Expirada,Total de Ações,Indicador 6
Unidade_Gestora,,,
ADH,60,65,0.923077
AGESPISA,15,25,0.600000
CDTER,0,1,0.000000
CFLP,12,15,0.800000


In [97]:
# INDICADOR 8


ind_08 = df_simo[df_simo['start_date'] >= pd.to_datetime(data_ref)].groupby(['Unidade_Gestora']).apply(lambda x: pd.Series({
    'Ações Planejadas':  (x['Estagio_atual'] == "ATIVIDADES PREPARATÓRIAS").sum(),
    'Ações Concluídas':  (x['Status'] == "Concluído").sum(), # len(x)  # ou x['Status'].count()
}))

ind_08['Indicador 8'] = ind_08['Ações Planejadas'] / ind_08['Ações Concluídas']

ind_08.head(100)

,Ações Planejadas,Ações Concluídas,Indicador 8
Unidade_Gestora,,,
ADAPI,1,0,inf
ADH,7,14,0.500000
AGESPISA,0,4,0.000000
CDSOL,0,15,0.000000
CDTER,80,0,inf
CFLP,9,5,1.800000
COORD. MODERNIZAÇÃO,0,13,0.000000
CPCPR,0,1,0.000000
DER,114,117,0.974359
